In [11]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [7]:
class_name = "Work"
count = CLIENT.query.aggregate(class_name).with_meta_count().do()
jprint(count)

<IPython.core.display.JSON object>

In [8]:
jprint(
    CLIENT.query
        .get(class_name="Work", properties=[])
        .with_limit(20)
        .with_near_text({"concepts": ["pictures of musicians"]})
        .with_group_by(["collection"], groups=10, objects_per_group=2)
        .with_additional([
            """
            group {
                id
                groupedBy {
                    value
                }
                count
                hits {
                    identifier
                    title
                    subject
                }
            }
            """
        ])
        .do()
)

<IPython.core.display.JSON object>

In [15]:
work=(
    CLIENT.query
        .get(class_name="Work", properties=["identifier", "title"])
        .with_limit(1)
        .with_additional(["vector"])
        .do()
)
vector=work["data"]["Get"]["Work"][0]["_additional"]["vector"]
jprint(work, expanded=False)

<IPython.core.display.JSON object>

In [16]:
result=(
    CLIENT.query
        .get(class_name="Work", properties=["identifier", "title"])
        .with_near_vector({"vector": vector})
        .with_limit(100)
        .do()
)
jprint(result)

<IPython.core.display.JSON object>